# Exercise Set 2

### True or False

**Q1.** Suppose the solar forecast for the week totaled more energy than the total demand. In this case it's guaranteed that no energy will be bought from the grid.
- True
- `False`

### Formualtion and Coding

In [ ]:
%pip install gurobipy

import gurobipy as gp
from gurobipy import GRB

batteries = ["Battery0", "Battery1"]
time_periods = range(180)
capacity = {"Battery0": 60, "Battery1": 80} 
p_loss = {"Battery0": 0.95, "Battery1": 0.9} 
initial = {"Battery0": 20, "Battery1": 30} 

m = gp.Model() 

# flow_in = m.addVars(batteries, time_periods, name="flow_in") 
# flow_out = m.addVars(batteries, time_periods, name="flow_out")
# grid = m.addVars(time_periods, name="grid")
# state = m.addVars(batteries, time_periods, name="state") 
# gen = m.addVars(time_periods, name="gen")

Feel free to use the commented code above to copy and paste.

**Q2.** Write a line of code that sets the objective coefficients for the energy storage problem (minimizing grid electricity purchased) using the `addVars()` command. Check your code by writing to a file named `model.lp` 

In [ ]:
#### gurobipy code
grid = m.addVars(time_periods, obj = 1, name="grid")
m.write('model.lp')

**Q3.** [Depth of discharge](https://en.wikipedia.org/wiki/Depth_of_discharge) refers to the percentage a battery's level is below its capacity. To help maintain the batteries, a restriction is put into place that says the depth of discharge for **both batteries combined** cannot exceed 70% of the **total capacity**. Formulate constraints (write the inequalities algebraically) that model this and write code in gurobypi.

**Hints:** Make sure your generator expression is in parenthesis if you get an error and consider the initial state a few cells above.

Formulation:
$$
\begin{align*}
\sum_bs_{b,t} \ge (1-0.7)*\sum_bc_b, \quad \forall t \in T
\end{align*}
$$

In [ ]:
#### gurobipy code
state = m.addVars(batteries, time_periods, name="state") 
depth_of_discharge = m.addConstrs((gp.quicksum(state[b,t] for b in batteries) >= 0.3*sum(capacity[b] for b in batteries) for t in time_periods), name = 'depth_restriction')

**Q4.** At the start of each day (other than the first) it is required that each battery be at least 40% full. The time periods for the start of each day are 30, 60, 90, 120, and 150. Formulate these constraints and write gurpbipy code. Do not worry about the solar forecast nor the contrstaint in **Q3**.

Formulation:
$$
\begin{align*}
s_{b,t} \ge 0.4*c_b, \quad \forall t \in [30,60,90,120,150]
\end{align*}
$$

In [ ]:
#### gurobipy code
m = gp.Model() # define the model again to make sure it is reset
state = m.addVars(batteries, time_periods, name="state") 
start_of_day = m.addConstrs((state[b,t] >= 0.4*capacity[b] for b in batteries for t in [30,60,90,120,150]), name = 'start_of_day') #in time_periods if t

### Modeling with Binary Variables

You have five light bulbs. Let $y_i = 1$ if lighbulb $i$ is on and $y_i = 0$ if it is off, for$\space i \in \{1,2,3,4,5\}$. Model the following statements using binary variables and $\le,\space\ge,\space=$. All questions are independent of each other. 

**Q4.** No more than three of the lightbulbs can be on.
$$
\begin{align*}
\sum_i y_i \le 3
\end{align*}
$$

**Q5.** Lightbulb 2 or ligthbulb 3 must be on.
$$
\begin{align*}
y_2 + y_3 \ge 1
\end{align*}
$$

**Q6.** If lightbulb 1 is on, then lightbulb 5 is also on. 
$$
\begin{align*} 
y_1 \le y_5
\end{align*}
$$

**Q7.** If lightbulb 1 is off, then lightbulb 3 is on. 
$$
\begin{align*} 
1-y_1 \le y_3
\end{align*}
$$

**Q8.** If lightbulb 1 is off, then lightbulb 2 is off. 
$$
\begin{align*} 
1-y_1 &\le 1-y_2 \\
y_1 &\ge y_2
\end{align*}
$$

**Q9.** Lightbulb 2 and lightbulb 4 are in opposite on/off states.
$$
\begin{align*} 
y_2 =& 1-y_4 \\
y_2 + y_4 =& 1
\end{align*}
$$

**Q10.** If any of lightbulbs 1 through 4 are on, then lighbtulb 5 is also on. 
$$
\begin{align*}
y_1 + y_2 + y_3 + y_4 \le 4y_5
\end{align*}
$$